In [70]:
import os
import pandas as pd
import joblib
from sklearn.pipeline import make_pipeline
import numpy as np
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from category_encoders import MEstimateEncoder
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [65]:
class Dataset:
    def __init__(self, input_csv, submission_csv='', is_val=False,train_dataset=''):
        '''
        d1~d1913為特徵，共1913天，原始特徵提供。
        d1914~1941為驗證，共28天，應該是比賽結束後釋出。
        d1942~d1966為測試，共28天，隱藏起來的資料範圍。


        '''
        self.is_val=is_val

        self.train_dataset=train_dataset

        self.sales = pd.read_csv(input_csv,index_col=0) # 讀取資料

        self.id, self.x=self._preprocess_feature(self.sales)
        if not is_val:
            self.y=self.preprocess_target(self.sales,self.id)
            self.X_train, self.X_local_val, self.y_train, self.y_local_val = train_test_split(self.x, self.y, test_size=0.2, random_state=42)
        else:
            self.sales = pd.read_csv(submission_csv,index_col=0)
        # self._sales_processed = (
        #     sales
        #     .drop(["警示編號", "監控層級", "觸發說明"], axis=1)
        #     .assign(**sales.資料日期.dt.isocalendar(),
        #             month=sales.資料日期.dt.month,
        #             quarter=sales.資料日期.dt.quarter,
        #             hfy=sales.資料日期.dt.quarter.isin([3, 4])+1,
        #             bigMon=sales.資料日期.dt.month.isin([1, 3, 5, 7, 8, 10, 12]).astype(int))  # 將sales整理出更多的特徵資料
        # )
        # self.txn_c = txn[txn['tran_type'] == 'DEBIT'].drop("tran_type", axis=1)
        # self.txn_d = txn[txn['tran_type'] ==
        #                  'CREDIT'].drop("tran_type", axis=1)
        # self.txn_t = txn[txn['tran_type'] == 'TXN'].drop("tran_type", axis=1)

        # self._txn_processed = txn  # _get_data_from_sql整理出的txn資料
        # self._risk_processed = risk  # _get_data_from_sql整理出的risk資料
        # self._info_processed = info  # _get_data_from_sql整理出的info資料
        # self.ac = ac  # _get_data_from_sql從VW_NP_FSC_PARTY_ACCOUNT_BRIDGE整理出的客帳戶對照

        # self.train = self._get_data("train")  # VIP訓練資料
        # self.val = self._get_data("val")  # VIP驗證資料
        # self.test = self._get_data("test")  # VIP當天資料

        # self.seg_mapping = self._get_cust_seg_def()


    def _preprocess_feature(self, data):
        #target encoding feature
        if not self.is_val:
            x = data.iloc[:,:-28]
            object_columns = x.select_dtypes(include='object').columns
            self.encoder1 = TargetEncoder()
            
            self.encoder1.fit(x[object_columns], data.iloc[:,-28])
            target_encoder_x = self.encoder1.transform(x[object_columns])
            non_object_columns = x.select_dtypes(exclude='object').columns
            combined_df = pd.concat([target_encoder_x, x[non_object_columns]], axis=1)
        else:
            x = data
            object_columns = x.select_dtypes(include='object').columns
            target_encoder_x = self.train_dataset.encoder1.transform(x[object_columns])
            non_object_columns = x.select_dtypes(exclude='object').columns
            combined_df = pd.concat([target_encoder_x, x[non_object_columns]], axis=1)
        
        return [x for x in data.index],combined_df
    
    def preprocess_target(self, data, index):
        return data.loc[index].iloc[:,-28:]

In [66]:
train_data=Dataset(input_csv='..\input\sales_train_evaluation.csv', is_val=False)
val_data=Dataset(input_csv='..\input\sales_train_validation.csv', is_val=True, train_dataset=train_data)

In [67]:
train_data.X_train

,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,,,,,,,,,,,,,,,,,,,,,
FOODS_3_164_CA_4_evaluation,0.983928,2.205954,1.806054,0.907511,1.436865,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
HOBBIES_1_198_TX_1_evaluation,1.037716,0.814663,0.669381,1.008855,1.165628,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
FOODS_1_084_CA_3_evaluation,1.118399,1.315278,1.806054,2.085602,1.436865,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
FOODS_2_180_CA_4_evaluation,1.252869,1.245477,1.806054,0.907511,1.436865,0,0,0,0,0,...,2,0,1,2,2,0,0,1,0,1
HOUSEHOLD_2_121_TX_1_evaluation,0.930140,0.347767,0.865138,1.008855,1.165628,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_137_WI_3_evaluation,1.091504,2.205954,1.806054,1.070187,1.159615,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2,0
FOODS_3_117_CA_2_evaluation,1.629387,2.205954,1.806054,1.287635,1.436865,15,5,5,5,9,...,2,1,1,0,1,1,1,0,4,2
HOUSEHOLD_1_302_CA_1_evaluation,1.010822,1.365977,0.865138,1.466710,1.436865,0,3,0,1,0,...,3,1,1,0,0,0,0,0,2,0


In [69]:
class Models:
    def __init__(self, is_load=False):
        '''


        '''
        
        package_dir = os.path.dirname(os.path.abspath(os.getcwd()))
        self.model_path = os.path.join(package_dir,'models')
        os.makedirs(self.model_path, exist_ok=True)
        
        if is_load:
            self._load_models()
        else:
            self._set_28_models()

    # def _create_pipe(self, X_train):
    #     return make_pipeline(
    #         TargetEncoder(cols=X_train.columns[X_train.dtypes == 'object'],
    #                       handle_missing='return_nan'),
    #         SimpleImputer(missing_values=np.nan, strategy='mean'),
    #         XGBRegressor()
    #     )
    def _load_models(self):
        
        self.models = [joblib.load(os.path.join(self.model_path, fn))
                        for fn in sorted(os.listdir(self.model_path))]

    def _set_28_models(self):
        self.models=[]
        for i in range(28):
            self.models.append(XGBRegressor())

    def _train_28_models(self, X_train, X_local_val, y_train, y_local_val):
        for i in range(28):
            
            # from IPython import embed
            # embed()
            self.models[i].fit(X_train, y_train.iloc[:,i])
            self.models[i]
            y_pred = self.models[i].predict(X_local_val)
            # 計算均方根誤差
            mse = mean_squared_error(y_local_val.iloc[:,i], y_pred)
            rmse = np.sqrt(mse)
            print("mse:",mse,"     Root Mean Squared Error:", rmse)
            joblib.dump(self.models[i], os.path.join(
                self.model_path, f"model_day{str(i).zfill(2)}.pkl"), compress=3)
    def models_predict(self, X_val:pd.DataFrame):

        output = X_val.iloc[:,0:1]
        for i in range(28):
            val_data_tmp_y = models.models[i].predict(val_data.x)
            output[f'F{i+1}'] = val_data_tmp_y

        output=output.iloc[:,1:]


SyntaxError: incomplete input (804474586.py, line 49)

In [54]:
models=Models()

In [55]:
models._train_28_models(train_data.X_train,train_data.X_local_val,train_data.y_train,train_data.y_local_val)

mse: 0.007360132586005319      Root Mean Squared Error: 0.08579121508642548
mse: 2.884340229434133      Root Mean Squared Error: 1.698334545793064
mse: 4.496504046329596      Root Mean Squared Error: 2.1204961792772927
mse: 3.1544978010093465      Root Mean Squared Error: 1.7760905948203618
mse: 3.931476469660121      Root Mean Squared Error: 1.9827951154015184
mse: 4.974971632245646      Root Mean Squared Error: 2.230464443169997
mse: 6.29945874331774      Root Mean Squared Error: 2.5098722563743636
mse: 4.56170679615956      Root Mean Squared Error: 2.135815253283757
mse: 4.914010140666207      Root Mean Squared Error: 2.2167566715059657
mse: 4.5730248471489165      Root Mean Squared Error: 2.13846319752034
mse: 3.64683872878814      Root Mean Squared Error: 1.9096697957469349
mse: 4.91317258110943      Root Mean Squared Error: 2.2165677479178094
mse: 6.6166906507267385      Root Mean Squared Error: 2.572292878100536
mse: 4.845538044677952      Root Mean Squared Error: 2.201258286680

In [58]:
output = val_data.x.iloc[:,0:1]
for i in range(28):
    val_data_tmp_y = models.models[i].predict(val_data.x)
    output[f'F{i+1}'] = val_data_tmp_y

output=output.iloc[:,1:]

In [62]:
the_result=pd.concat([val_data.sales.iloc[:,0], output], axis=1)

In [63]:
the_result.to_csv('first_result.csv',index=False)

In [46]:
models.models[0].predict(train_data.X_local_val)

array([9.9999541e-01, 8.9999714e+00, 1.9999998e+00, ..., 9.9999541e-01,
       4.2390134e-06, 4.2390134e-06], dtype=float32)

In [49]:
train_data.y_local_val.iloc[:,0]

2404     1
7576     9
8731     2
4756     1
15509    0
        ..
8870     0
1423     0
27492    1
27020    0
21011    0
Name: d_1914, Length: 6098, dtype: int64

In [ ]:
mse = mean_squared_error(y_local_val[:,i], y_pred)

In [ ]:
models.models[0].predict(train_data.X_local_val)

In [42]:
train_data.y_train.iloc[:,0]


11535    0
12387    0
7792     0
11153    2
13413    0
        ..
29802    0
5390     0
860      1
15795    0
23654    1
Name: d_1914, Length: 24392, dtype: int64

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A       3 non-null      int64 
 1   B       3 non-null      object
 2   C       3 non-null      bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 179.0+ bytes


In [40]:
import pandas as pd

# 创建示例DataFrame
data = {'A': [1, 2, 3], 'B': ['foo', 'bar', 'baz'], 'C': [True, False, True]}
df = pd.DataFrame(data)

# 获取所有物件类型的列
object_columns = df.select_dtypes(exclude='object').columns

# 打印列名
print(df[object_columns])
# 这将打印出DataFrame中所有物件类型的列名。你可以根据需要使用这些列名进行进一步的操作，例如筛选特定的列或进行其他数据处理。








   A      C
0  1   True
1  2  False
2  3   True


In [33]:
# for x in train_data.sales.index:
#     print(x)

In [8]:


class model:
    def __init__(self, load=False):


        package_dir = os.path.dirname(os.path.abspath(__file__))
        self.model_path = os.path.join(
            package_dir, f'./models/')

        os.makedirs(self.model_path, exist_ok=True)

        if load:
            self.models = [joblib.load(os.path.join(self.model_path, fn))
                           for fn in sorted(os.listdir(self.model_path))]
        else:
            for fn in os.listdir(self.model_path):
                os.remove(os.path.join(self.model_path, fn))
            self.models = []
            self._train_inference = None
            self._train_27_models()

    def _create_pipe(self, X_train):
        return make_pipeline(
            TargetEncoder(cols=X_train.columns[X_train.dtypes == 'object'],
                          handle_missing='return_nan'),
            SimpleImputer(missing_values=np.nan, strategy='mean'),
            XGBRegressor()
        )
    def _train_27_models(self):
        for i in range(1, 28):
            train = self._create_label("train", i)
            x_train, y_train = train.drop(
                self.cols_dropped, axis=1), train.警示數量
            # from IPython import embed
            # embed()
            model = self._create_pipe(x_train)
            model.fit(x_train, y_train)
            joblib.dump(model, os.path.join(
                self.model_path, f"{datetime.now().date()}_w{str(i).zfill(2)}.pkl"), compress=3)
            self.models.append(model)

